In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN, Dropout
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("../data/data/합천_댐기상종합_forTrain.csv", index_col=0)
df.index = pd.to_datetime(df.index)

In [10]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [11]:
df_scaled = pd.DataFrame(data=scaled_data, columns=df.columns, index=df.index.values)
df_scaled.head()

,저수량(현재),전일방류량(본댐),당일유입량,전일유입량,1일후유입량,2일후유입량,홍수기,sin_day_of_week,cos_day_of_week,sin_month,cos_month,sin_week_of_year,cos_week_of_year,기온(°C),강수량(mm),지면온도(°C),습도(%)
2000-01-01,0.582062,0.017042,0.002319,0.003900,0.002477,0.002003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.347890,0.000000,0.209110,0.553127
2000-01-02,0.580077,0.013976,0.002477,0.002319,0.002003,0.003215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.421855,0.000000,0.239237,0.576004
2000-01-03,0.576244,0.015391,0.002003,0.002477,0.003215,0.004217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.302154,0.000000,0.176780,0.462633
2000-01-04,0.573911,0.016099,0.003215,0.002003,0.004217,0.003373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.273040,0.000000,0.168390,0.474835
2000-01-05,0.570909,0.014860,0.004217,0.003215,0.003373,0.003110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.345530,0.005805,0.209068,0.546518


In [9]:
df_scaled.columns

Index(['저수량(현재)', '전일방류량(본댐)', '당일유입량', '전일유입량', '1일후유입량', '2일후유입량', '홍수기',
       'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month',
       'sin_week_of_year', 'cos_week_of_year', '기온(°C)', '강수량(mm)', '지면온도(°C)',
       '습도(%)'],
      dtype='object')

In [17]:
df_scaled.drop(['전일유입량','1일후유입량','2일후유입량'], axis=1, inplace=True)

In [18]:
train = df_scaled.loc['2000-01-01':'2021-01-01']
test = df_scaled.loc['2021-01-01':]

In [19]:
train_y = train[['당일유입량']]
test_y = test[['당일유입량']]

In [20]:
train_scaled = train.values
test_scaled = test.values
train_scaled_y = train_y.values
test_scaled_y = test_y.values

In [21]:
# 훈련 데이터셋에 대해 X와 y를 분리하고 3일 단위로 묶기
def split_sequence(sequence,sequence_y, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(sequence)):
        # 인덱스를 기준으로 입력 시퀀스와 출력 시퀀스를 나누기
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix], sequence_y[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# 훈련 및 테스트 데이터셋 생성
n_steps_in = 7
n_steps_out = 3
X_train, y_train = split_sequence(train_scaled,train_scaled_y, n_steps_in, n_steps_out)
X_test, y_test = split_sequence(test_scaled,test_scaled_y, n_steps_in, n_steps_out)

In [23]:
from tensorflow.keras.optimizers.legacy import Adam

learning_rates = [0.001]
units = [64]
dropouts = [0.2]
epochs = [1000]

In [24]:
timesteps = 7
eval = pd.DataFrame(columns=['lr', 'Unit', 'dropout', 'epoch','RMSE', 'MAE', 'MSE', 'PBIAS', 'NSE'])
for lr in learning_rates:
  optimizer = Adam(learning_rate=lr)
  for u in units:
    for d in dropouts:
      for e in epochs:
        print("======================================================================================")
        print(f"units: {u}, dropout: {d}, epoch: {e}")

        model = Sequential()
        model.add(SimpleRNN(u, input_shape=(timesteps, X_train.shape[2]), return_sequences=True))
        model.add(Dropout(d)) 
        model.add(SimpleRNN(int(u/2)))
        model.add(Dropout(d))
        model.add(Dense(3))
        model.compile(loss='mse', optimizer=optimizer)
        model.fit(X_train, y_train, epochs=e, batch_size=32, verbose=0)
        
        y_pred = model.predict(X_test)
        y_test = y_test.reshape(y_test.shape[0], y_test.shape[1])

        #RMSE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print('RMSE:', rmse, end="\t")
        #MAE
        mae = mean_absolute_error(y_test, y_pred)
        print('MAE:', mae, end="\t")
        #MSE
        mse = mean_squared_error(y_test.reshape(-1), y_pred.reshape(-1))
        print("MSE:", mse, end="\t")

        # PBIAS 계산
        diff = np.sum(y_pred - y_test)
        sum_observed = np.sum(y_test)
        pbias = (diff / sum_observed) * 100
        print('PBIAS:', pbias, end="\t")

        # NSE 계산
        diff_pred_obs = np.sum((y_pred - y_test) ** 2)
        diff_obs_mean = np.sum((y_test - np.mean(y_test)) ** 2)
        nse = 1 - (diff_pred_obs / diff_obs_mean)
        print('NSE:', nse, end="\t")
        print()

        array = [lr,u,d,e,rmse, mae, mse, pbias, nse]
        eval.loc[eval.shape[0]] = array

        mae = np.mean(np.abs(y_test - y_pred), axis=0)
        mse = np.mean(np.square(y_test - y_pred), axis=0)
        plt.figure(figsize=(10, 6))
        plt.plot(mae, label="MAE")
        plt.plot(mse, label="MSE")
        plt.xlabel("Time Step")
        plt.ylabel("Error")
        plt.legend()
        plt.savefig(f"images/1_units_{u} dropout_{d} epoch_{e} lr_{lr}.png")
        plt.close()

        plt.figure(figsize=(16, 8))
        plt.plot(y_test, label="Actual")
        plt.plot(y_pred, label="Predicted")
        plt.legend()
        plt.savefig(f"images/2_units_{u} dropout_{d} epoch_{e} lr_{lr}.png")
        plt.close()

        y_test_df = pd.DataFrame(y_test)
        y_pred_df = pd.DataFrame(y_pred)

        test_dates = df_scaled.index[n_steps_in+len(X_train)+n_steps_out-1:n_steps_in+len(X_train)+n_steps_out-1+len(X_test)]
        plt.figure(figsize=(16, 8))
        plt.plot(test_dates, y_test_df.iloc[:, 0], label='Actual')
        plt.plot(test_dates, y_pred_df.iloc[:, 0], label='Predicted', linestyle='--')
        plt.legend()
        plt.savefig(f"images/3_units_{u} dropout_{d} epoch_{e} lr_{lr}.png")
        plt.close()

        plt.figure(figsize=(16, 8))
        plt.plot(test_dates, y_test_df.iloc[:, 1], label='Actual')
        plt.plot(test_dates, y_pred_df.iloc[:, 1], label='Predicted', linestyle='--', color="#C71585")
        plt.legend()
        plt.savefig(f"images/4_units_{u} dropout_{d} epoch_{e} lr_{lr}.png")
        plt.close()

        plt.figure(figsize=(16, 8))
        plt.plot(test_dates, y_test_df.iloc[:, 2], label='Actual')
        plt.plot(test_dates, y_pred_df.iloc[:, 2], label='Predicted', linestyle='--', color="#FA8072")
        plt.legend()
        plt.savefig(f"images/5_units_{u} dropout_{d} epoch_{e} lr_{lr}.png")
        plt.close()

        # 날짜 범위 생성
        train_dates = df_scaled.index[n_steps_in:n_steps_in+len(X_train)]
        test_dates = df_scaled.index[n_steps_in+len(X_train)+n_steps_out-1:n_steps_in+len(X_train)+n_steps_out-1+len(X_test)]

        plt.figure(figsize=(10, 6))
        plt.plot(train_dates, y_train[:, 0], label='Actual (Train)', color='blue')
        plt.plot(test_dates, y_test[:, 0], label='Actual (Test)', color='green')
        plt.plot(test_dates, y_pred[:, 0], label='Predicted', color='red')
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title('Actual vs Predicted')
        plt.gcf().autofmt_xdate()
        plt.legend()
        plt.savefig(f"images/6_units_{u} dropout_{d} epoch_{e} lr_{lr}.png")
        plt.close()

eval.to_csv('RNN_result.csv')

units: 64, dropout: 0.2, epoch: 1000
26/26 [==============================] - 0s 958us/step
RMSE: 0.014018048074355704	MAE: 0.007487649703540148	MSE: 0.0001965056718149477	PBIAS: 119.1331889419209	NSE: -0.08443190921472477	
